# 强化学习算法

强化学习本质是训练一个agent，存在一个environment，agent输出操作给环境，环境执行后返回状态和奖励，agent接受后生成新的操作

比如：降落游戏，agent是飞船，有四种输出，每种对应控制飞船推进器，环境是一个雅达利游戏，接受操作控制飞船，如果飞船降落给100分，如果坠毁或者其他奖励为负数

目标就是训练这个agent，通过这种奖励机制，使得能够完成降落

## 强化学习工作类型

两种：

1. 关卡式的，比如飞船降落，每次坠毁就代表关卡结束
2. 连续式的，比如操作股票，没有停止的时候

## 强化学习算法/训练类型

agent有两种：

1. 基于价值函数的，输入：当前这个状态+不同操作，输出未来价值最高的操作
2. 确定性的，输入：当前状态，输出操作，或者操作的概率分布（a操作：80%，b操作20%）

对应的agent的训练也有两种：

1. 基于策略训练：直接训练策略，底层可能是个MLP，输入状态，输出行为
2. 基于价值的训练：训练的是价值函数，找到一个函数，初始可能啥都不知道，训练完了以后，输入环境和操作，输出价值

## 深度强化学习算法

基于神经网络来做RL

而且一般训练的时候，游戏环境是一次开多个游戏，每个游戏的状态（或者观察）压缩在一个向量

环境有很多种：

    1. mlagents 是可以不开启渲染的情况下和unity游戏交互
    2. gymnasium 提供了各种游戏环境，可以不开启渲染

agent也有库有实现：

    1. stable_baselines3 实现了各种模型算法，包括PPO等

模型和hf的库，用于上传模型：

    1. huggingface_hub
    2. huggingface_sb3：专门加载和保存sb3的算法模型

基于价值函数的算法：Q-learning

## 简单DEMO

### 配置环境

In [ ]:
!apt install swig cmake
!pip install shimmy
!pip install -r https://raw.githubusercontent.com/huggingface/deep-rl-class/main/notebooks/unit1/requirements-unit1.txt
!sudo apt-get update
!apt install python3-opengl
!apt install ffmpeg
!apt install xvfb
!pip3 install pyvirtualdisplay

In [ ]:
# 使用noetbook时，配置好后要重启内核
import os
os.kill(os.getpid(), 9)

In [ ]:
# Virtual display
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

import gymnasium

from huggingface_sb3 import load_from_hub, package_to_hub
from huggingface_hub import (
    notebook_login,
)  # To log to our Hugging Face account to be able to upload models to the Hub.

from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

## 环境库小demo


In [ ]:
import gymnasium as gym

# First, we create our environment called LunarLander-v2
env = gym.make("LunarLander-v2")

# Then we reset this environment
observation, info = env.reset()

for _ in range(20):
    # Take a random action
    action = env.action_space.sample()
    print("Action taken:", action)

    # Do this action in the environment and get
    # next_state, reward, terminated, truncated and info
    observation, reward, terminated, truncated, info = env.step(action)

    # If the game is terminated (in our case we land, crashed) or truncated (timeout)
    if terminated or truncated:
        # Reset the environment
        print("Environment is reset")
        observation, info = env.reset()

env.close()


In [ ]:
# 创建场景并且查看在这个游戏下，环境是什么样的，操作有哪些取值
# We create our environment with gym.make("<name_of_the_environment>")
env = gym.make("LunarLander-v2")
env.reset()
print("_____OBSERVATION SPACE_____ \n")
print("Observation Space Shape", env.observation_space.shape)
print("Sample observation", env.observation_space.sample())  # Get a random observation

print("\n _____ACTION SPACE_____ \n")
print("Action Space Shape", env.action_space.n)
print("Action Space Sample", env.action_space.sample())  # Take a random action

In [ ]:
# 如何一次创建n个场景，并且将这些场景合并成一个向量
# action的形状通常是(16, act_dim)（act_dim是动作空间的维度），表示每个环境执行的动作。
# 例如，如果动作是控制飞船推进器的开关（假设动作空间维度act_dim = 4），那么action就是一个形状为(16, 4)的数组，用于同时对 16 个环境中的飞船进行控制
env = make_vec_env("LunarLander-v2", n_envs=16)

# 创建demo真正的游戏场景，飞船降落
# Create environment
env = gym.make('LunarLander-v2')

# Instantiate the agent
model = PPO('MlpPolicy', env, verbose=1)
# Train the agent
model.learn(total_timesteps=int(2e5))
# 创建使用PPO算法的agent
# We added some parameters to accelerate the training
model = PPO(
    policy="MlpPolicy",
    env=env,
    n_steps=1024,
    batch_size=64,
    n_epochs=4,
    gamma=0.999,
    gae_lambda=0.98,
    ent_coef=0.01,
    verbose=1,
)

# 训练agent
# SOLUTION
# Train it for 1,000,000 timesteps
model.learn(total_timesteps=1000000)
# Save the model
model_name = "ppo-LunarLander-v2"
model.save(model_name)

# 评估agent
# @title
eval_env = Monitor(gym.make("LunarLander-v2"))
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True)
print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

## 上传模型

In [ ]:
# 登陆自己的hf账号，得到访问的key (https://huggingface.co/settings/tokens) 
# 如果是notebook情况下
notebook_login()
!git config --global credential.helper store

# 如果是其他情况，在命令行执行：huggingface-cli login

# 执行上传
import gymnasium as gym

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.env_util import make_vec_env

from huggingface_sb3 import package_to_hub

# PLACE the variables you've just defined two cells above
# Define the name of the environment
env_id = "LunarLander-v2"

# TODO: Define the model architecture we used
model_architecture = "PPO"

## Define a repo_id
## repo_id is the id of the model repository from the Hugging Face Hub (repo_id = {organization}/{repo_name} for instance ThomasSimonini/ppo-LunarLander-v2
## CHANGE WITH YOUR REPO ID
repo_id = "Segment139/ppo-LunarLander-v2"  # Change with your repo id, you can't push with mine 😄

## Define the commit message
commit_message = "Upload PPO LunarLander-v2 trained agent"

# Create the evaluation env and set the render_mode="rgb_array"
eval_env = DummyVecEnv([lambda: Monitor(gym.make(env_id, render_mode="rgb_array"))])

# PLACE the package_to_hub function you've just filled here
package_to_hub(
    model=model,  # Our trained model
    model_name=model_name,  # The name of our trained model
    model_architecture=model_architecture,  # The model architecture we used: in our case PPO
    env_id=env_id,  # Name of the environment
    eval_env=eval_env,  # Evaluation Environment
    repo_id=repo_id,  # id of the model repository from the Hugging Face Hub (repo_id = {organization}/{repo_name} for instance ThomasSimonini/ppo-LunarLander-v2
    commit_message=commit_message,
)

## 加载其他人的模型

In [ ]:
from huggingface_sb3 import load_from_hub

repo_id = "Classroom-workshop/assignment2-omar"  # The repo_id
filename = "ppo-LunarLander-v2.zip"  # The model filename.zip

# When the model was trained on Python 3.8 the pickle protocol is 5
# But Python 3.6, 3.7 use protocol 4
# In order to get compatibility we need to:
# 1. Install pickle5 (we done it at the beginning of the colab)
# 2. Create a custom empty object we pass as parameter to PPO.load()
custom_objects = {
    "learning_rate": 0.0,
    "lr_schedule": lambda _: 0.0,
    "clip_range": lambda _: 0.0,
}

checkpoint = load_from_hub(repo_id, filename)
model = PPO.load(checkpoint, custom_objects=custom_objects, print_system_info=True)

# 评估模型
# @title
eval_env = Monitor(gym.make("LunarLander-v2"))
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True)
print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")